In [21]:
import numpy as np
import scipy.special as spl

In [22]:
import classes

In [23]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("RN50", device=device)

text = clip.tokenize(classes.CLASSES).to(device)

with torch.no_grad():
    text_features = model.encode_text(text).cpu()

In [24]:
print(len(classes.CLASSES))
print(len(text_features))

1000
1000


In [11]:
# Adjacency matrix
from scipy import spatial

adj_matrix = np.zeros((1000,1000))

for i in range(1000):
    for j in range(1000):
        adj_matrix[i][j] = 1 - spatial.distance.cosine(text_features[i], text_features[j])
        
adj_matrix

array([[1.        , 0.74414062, 0.62646484, ..., 0.59716797, 0.64892578,
        0.62353516],
       [0.74414062, 1.        , 0.69140625, ..., 0.61132812, 0.71679688,
        0.72412109],
       [0.62646484, 0.69140625, 1.        , ..., 0.5390625 , 0.63378906,
        0.64746094],
       ...,
       [0.59716797, 0.61132812, 0.5390625 , ..., 1.        , 0.59423828,
        0.59228516],
       [0.64892578, 0.71679688, 0.63378906, ..., 0.59423828, 1.        ,
        0.71386719],
       [0.62353516, 0.72412109, 0.64746094, ..., 0.59228516, 0.71386719,
        1.        ]])

In [13]:
from sklearn.cluster import SpectralClustering

sc = SpectralClustering(10, affinity='precomputed', n_init=100, assign_labels='discretize')
out = sc.fit_predict(adj_matrix)  

In [19]:
clusters = {}
for i, cluster in enumerate(out):
    if cluster not in clusters:
        clusters[cluster] = []
        
    clusters[cluster].append(classes.CLASSES[i])

In [20]:
clusters

{9: ['tench',
  'goldfish',
  'rooster',
  'brambling',
  'goldfinch',
  'junco',
  'bulbul',
  'magpie',
  'chickadee',
  'American dipper',
  'kite (bird of prey)',
  'vulture',
  'newt',
  'chameleon',
  'scorpion',
  'tick',
  'black grouse',
  'quail',
  'partridge',
  'macaw',
  'coucal',
  'bee eater',
  'hornbill',
  'hummingbird',
  'jacamar',
  'toucan',
  'duck',
  'goose',
  'platypus',
  'wombat',
  'conch',
  'Dungeness crab',
  'rock crab',
  'crayfish',
  'black stork',
  'spoonbill',
  'bittern bird',
  'crane bird',
  'limpkin',
  'common gallinule',
  'bustard',
  'dunlin',
  'dowitcher',
  'oystercatcher',
  'king penguin',
  'killer whale',
  'dugong',
  'Papillon',
  'dingo',
  'dhole',
  'red fox',
  'grey fox',
  'cougar',
  'lion',
  'mongoose',
  'grasshopper',
  'praying mantis',
  'cottontail rabbit',
  'hare',
  'hamster',
  'porcupine',
  'marmot',
  'beaver',
  'common sorrel horse',
  'gazelle',
  'weasel',
  'mink',
  'badger',
  'proboscis monkey',
  '

In [38]:
final = {}
for i, cls in enumerate(classes.CLASSES):
    
    out = []
    for j in range(1000):
        val = round(1 - spatial.distance.cosine(text_features[i], text_features[j]),5)
        out.append(val)
    norm_out = []
    for j in out:
        norm_out.append(j/sum(out))
    inds = np.array(norm_out).argsort()[-3:][::-1]
    final[i] = out
    print(cls, ":", np.array(classes.CLASSES)[inds])


tench : ['tench' 'goldfish' 'duck']
goldfish : ['goldfish' 'duck' 'lion']
great white shark : ['great white shark' 'tiger shark' 'killer whale']
tiger shark : ['tiger shark' 'great white shark' 'tiger cat']
hammerhead shark : ['hammerhead shark' 'tiger shark' 'great white shark']
electric ray : ['electric ray' 'stingray' 'tiger shark']
stingray : ['stingray' 'electric ray' 'tiger shark']
rooster : ['rooster' 'hen' 'duck']
hen : ['hen' 'duck' 'fly']
ostrich : ['ostrich' 'ox' 'duck']
brambling : ['brambling' 'goldfinch' 'dunlin']
goldfinch : ['goldfinch' 'chickadee' 'junco']
house finch : ['house finch' 'goldfinch' 'indigo bunting']
junco : ['junco' 'dingo' 'duck']
indigo bunting : ['indigo bunting' 'house finch' 'European green lizard']
American robin : ['American robin' 'American black bear' 'prairie grouse']
bulbul : ['bulbul' 'bubble' 'fly']
jay : ['jay' 'fly' 'pig']
magpie : ['magpie' 'badger' 'killer whale']
chickadee : ['chickadee' 'hummingbird' 'junco']
American dipper : ['Americ

oystercatcher : ['oystercatcher' 'black stork' 'magpie']
pelican : ['pelican' 'duck' 'ostrich']
king penguin : ['king penguin' 'duck' 'baboon']
albatross : ['albatross' 'goose' 'ostrich']
grey whale : ['grey whale' 'grey wolf' 'great white shark']
killer whale : ['killer whale' 'great white shark' 'tiger shark']
dugong : ['dugong' 'tiger shark' 'polar bear']
sea lion : ['sea lion' 'lion' 'otter']
Chihuahua : ['Chihuahua' 'Pomeranian' 'pug']
Japanese Chin : ['Japanese Chin' 'King Charles Spaniel' 'Shih Tzu']
Maltese : ['Maltese' 'Brittany dog' 'Dalmatian']
Pekingese : ['Pekingese' 'Shih Tzu' 'Lhasa Apso']
Shih Tzu : ['Shih Tzu' 'Lhasa Apso' 'Tibetan Terrier']
King Charles Spaniel : ['King Charles Spaniel' 'English Springer Spaniel' 'Japanese Chin']
Papillon : ['Papillon' 'bulbul' 'collie']
toy terrier : ['toy terrier' 'Miniature Pinscher' 'Yorkshire Terrier']
Rhodesian Ridgeback : ['Rhodesian Ridgeback' 'Redbone Coonhound' 'Bullmastiff']
Afghan Hound : ['Afghan Hound' 'Scottish Deerhoun

Great Pyrenees dog : ['Great Pyrenees dog' 'Samoyed' 'Golden Retriever']
Samoyed : ['Samoyed' 'Great Pyrenees dog' 'Arctic fox']
Pomeranian : ['Pomeranian' 'Persian cat' 'Chihuahua']
Chow Chow : ['Chow Chow' 'lion' 'brown bear']
Keeshond : ['Keeshond' 'Norwegian Elkhound' 'Pekingese']
brussels griffon : ['brussels griffon' 'Shih Tzu' 'Pekingese']
Pembroke Welsh Corgi : ['Pembroke Welsh Corgi' 'Cardigan Welsh Corgi' 'Shetland Sheepdog']
Cardigan Welsh Corgi : ['Cardigan Welsh Corgi' 'Pembroke Welsh Corgi' 'Border Collie']
Toy Poodle : ['Toy Poodle' 'Miniature Poodle' 'Standard Poodle']
Miniature Poodle : ['Miniature Poodle' 'Toy Poodle' 'Standard Poodle']
Standard Poodle : ['Standard Poodle' 'Miniature Poodle' 'Toy Poodle']
Mexican hairless dog (xoloitzcuintli) : ['Mexican hairless dog (xoloitzcuintli)' 'Scottish Terrier'
 'Italian Greyhound']
grey wolf : ['grey wolf' 'coyote' 'grey fox']
Alaskan tundra wolf : ['Alaskan tundra wolf' 'grey wolf' 'Arctic fox']
red wolf or maned wolf : ['r

altar : ['altar' 'church' 'restaurant']
ambulance : ['ambulance' 'police van' 'fire truck']
amphibious vehicle : ['amphibious vehicle' 'recreational vehicle' 'race car']
analog clock : ['analog clock' 'digital clock' 'wall clock']
apiary : ['apiary' 'valley' 'bee']
apron : ['apron' 'military uniform' 'gown']
trash can : ['trash can' 'milk can' 'bucket']
assault rifle : ['assault rifle' 'rifle' 'missile']
backpack : ['backpack' 'messenger bag' 'purse']
bakery : ['bakery' 'butcher shop' 'grocery store']
balance beam : ['balance beam' 'parallel bars' 'hammer']
balloon : ['balloon' 'soccer ball' 'golf ball']
ballpoint pen : ['ballpoint pen' 'fountain pen' 'comic book']
Band-Aid : ['Band-Aid' 'comic book' 'toilet paper']
banjo : ['banjo' 'jay' 'rifle']
baluster / handrail : ['baluster / handrail' 'split-rail fence' 'parallel bars']
barbell : ['barbell' 'dumbbell' 'basketball']
barber chair : ['barber chair' 'barbershop' 'coffeemaker']
barbershop : ['barbershop' 'barber chair' 'shoe store']


fountain : ['fountain' 'fountain pen' 'stage']
fountain pen : ['fountain pen' 'fountain' 'letter opener']
four-poster bed : ['four-poster bed' 'infant bed' 'bookcase']
freight car : ['freight car' 'railroad car' 'garbage truck']
French horn : ['French horn' 'cornet' 'drum']
frying pan : ['frying pan' 'wok' 'gong']
fur coat : ['fur coat' 'trench coat' 'feather boa']
garbage truck : ['garbage truck' 'trash can' 'tow truck']
gas mask or respirator : ['gas mask or respirator' 'oxygen mask' 'mask']
gas pump : ['gas pump' 'vending machine' 'payphone']
goblet : ['goblet' 'tea cup' 'beer glass']
go-kart : ['go-kart' 'race car' 'sports car']
golf ball : ['golf ball' 'soccer ball' 'tennis ball']
golf cart : ['golf cart' 'sports car' 'golf ball']
gondola : ['gondola' 'ski' 'canoe']
gong : ['gong' 'drum' 'safe']
gown : ['gown' 'academic gown' 'military uniform']
grand piano : ['grand piano' 'upright piano' 'race car']
greenhouse : ['greenhouse' 'boathouse' 'barn']
radiator grille : ['radiator gril

piggy bank : ['piggy bank' 'pig' 'coffee mug']
pill bottle : ['pill bottle' 'wine bottle' 'beer bottle']
pillow : ['pillow' 'couch' 'coffee mug']
ping-pong ball : ['ping-pong ball' 'golf ball' 'tennis ball']
pinwheel : ['pinwheel' 'car wheel' 'spiral or coil']
pirate ship : ['pirate ship' 'sailboat' 'shipwreck']
drink pitcher : ['drink pitcher' 'coffee mug' 'water jug']
block plane : ['block plane' 'race car' 'slip-on shoe']
planetarium : ['planetarium' 'church' 'movie theater']
plastic bag : ['plastic bag' 'garbage truck' 'trash can']
plate rack : ['plate rack' 'plate' 'bookcase']
farm plow : ['farm plow' 'snowplow' 'tow truck']
plunger : ['plunger' 'hammer' 'ladle']
Polaroid camera : ['Polaroid camera' 'reflex camera' 'radio']
pole : ['pole' 'flagpole' 'rifle']
police van : ['police van' 'ambulance' 'semi-trailer truck']
poncho : ['poncho' 'cloak' 'plastic bag']
pool table : ['pool table' 'dining table' 'desk']
soda bottle : ['soda bottle' 'beer bottle' 'wine bottle']
plant pot : ['p

turnstile : ['turnstile' 'missile' 'missile']
typewriter keyboard : ['typewriter keyboard' 'computer keyboard' 'keyboard space bar']
umbrella : ['umbrella' 'parachute' 'tent']
unicycle : ['unicycle' 'tricycle' 'mountain bike']
upright piano : ['upright piano' 'grand piano' 'pipe organ']
vacuum cleaner : ['vacuum cleaner' 'clothes iron' 'coffeemaker']
vase : ['vase' 'water jug' 'coffee mug']
vaulted or arched ceiling : ['vaulted or arched ceiling' 'viaduct' 'dome']
velvet fabric : ['velvet fabric' 'red wine' 'vestment']
vending machine : ['vending machine' 'slot machine' 'gas pump']
vestment : ['vestment' 'T-shirt' 'military uniform']
viaduct : ['viaduct' 'valley' 'through arch bridge']
violin : ['violin' 'cello' 'electric guitar']
volleyball : ['volleyball' 'basketball' 'baseball']
waffle iron : ['waffle iron' 'clothes iron' 'coffeemaker']
wall clock : ['wall clock' 'analog clock' 'digital clock']
wallet : ['wallet' 'pencil case' 'mobile phone']
wardrobe : ['wardrobe' 'T-shirt' 'milita

In [39]:
import json

with open('/nethome/bdevnani3/flash1/cmo/imagenet_class_similarity.json', 'w') as f:
    json.dump(final, f)